In [1]:
weights = [0.9800796812749004, 1.020746887966805]

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, random_split
from PIL import Image
import os
from tqdm import tqdm

class CustomDataset(Dataset):
    def __init__(self, data_dir, result, transform):
        self.data_dir = data_dir
        self.result = result
        self.transform = transform
        self.images = []
        self.output = []
        # Iterate directory
        for file in os.listdir(data_dir):
            # check if current path is a file
            if os.path.isfile(os.path.join(data_dir, file)):
                img_path = os.path.join(data_dir, file)
                self.images.append(img_path)
                self.output.append(result)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
#         img = Image.open(self.images[idx])
        out = self.output[idx]
        with Image.open(self.images[idx]) as img:
            img = img.convert('RGB')
            if self.transform:
                img = self.transform(img)
#             img = img.astype('float32') / 255

        return img, out


In [12]:
# Define the transformations to be applied to the images and masks
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.RandomRotation(10),
    transforms.RandomVerticalFlip(0.5),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [13]:
from sklearn.model_selection import train_test_split
# Create the dataset for parasite
parasite_dataset = CustomDataset('C:\\Users\\Admin\\Coding-M-Tech\\AI Lab - ID5002W\\Assignment 7\\Datasets\\ch22m503\\parasite\\',1, transform)
# train_par_set, test_par_set = random_split(parasite_dataset, [328, 80])
# Create the dataset for uninfected
uninfected_dataset = CustomDataset('C:\\Users\\Admin\\Coding-M-Tech\\AI Lab - ID5002W\\Assignment 7\\Datasets\\ch22m503\\uninfected\\',0, transform)
# train_uni_set, test_uni_set = random_split(uninfected_dataset, [67, 20])
# train_t_set = train_par_set + train_uni_set
# test_t_set = test_par_set + test_uni_set
# train_t_set+=test_t_set
total_dataset = parasite_dataset + uninfected_dataset

#First normalizing the data by dividing each pixel value by 255. This will ensure that all the pixel values are between 0 and 1.
# total_dataset = total_dataset.astype('float32') / 255.
train_set, test_t_set = train_test_split(total_dataset, test_size=0.2, random_state=42)
train_t_set, val_t_set = train_test_split(train_set, test_size=0.2, random_state=42)

In [14]:
print(f"Shape of train Dataset {len(train_t_set)}")
print(f"Shape of test Dataset {len(test_t_set)}")
print(f"Shape of validate Dataset {len(val_t_set)}")


Shape of train Dataset 314
Shape of test Dataset 99
Shape of validate Dataset 79


In [15]:
# Use the datasets in your training and validation loops
train_loader = torch.utils.data.DataLoader(train_t_set, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_t_set, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_t_set, batch_size=64, shuffle=True)

In [16]:
# Define the CNN model
class CNN_Model(nn.Module):
    def __init__(self):
        super(CNN_Model, self).__init__()
        
        self.cd1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3)
        self.MP = nn.MaxPool2d(kernel_size=2, stride=2)
        self.cd2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3)
        self.fc1 = nn.Linear(in_features=32*14*14, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=2)
        
    def forward(self, x):
        x = self.MP(torch.relu(self.cd1(x)))
        x = self.MP(torch.relu(self.cd2(x)))
        x = x.view(-1, 32*14*14)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [23]:
# Initialize the CNN model and optimizer
model = CNN_Model()
opti = optim.Adam(model.parameters(), lr=0.001)
crit = nn.CrossEntropyLoss(weight=torch.Tensor(weights))

# Train the model
num_epochs = 36
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
for epoch in range(num_epochs):
    run_loss = 0.0
    run_acc = 0.0
    for i, (featur, labels) in enumerate(tqdm(train_loader)):
        featur = featur.to(device)
        labels = labels.to(device)
        opti.zero_grad()
        out = model(featur)
        loss = crit(out, labels)
        loss.backward()
        opti.step()
        run_loss += loss.item() * featur.size(0)
        _, preds = torch.max(out, 1)
        run_acc += torch.sum(preds == labels.data)
    epoch_loss = run_loss / len(train_t_set)
    epoch_acc = run_acc.double() / len(train_t_set)
    print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.4f}'.format(epoch+1, num_epochs, epoch_loss, epoch_acc))

100%|██████████| 5/5 [00:02<00:00,  1.86it/s]


Epoch [1/36], Loss: 0.6936, Accuracy: 0.5223


100%|██████████| 5/5 [00:01<00:00,  3.12it/s]


Epoch [2/36], Loss: 0.6376, Accuracy: 0.6465


100%|██████████| 5/5 [00:03<00:00,  1.48it/s]


...

Epoch [34/36], Loss: 0.0083, Accuracy: 1.0000


100%|██████████| 5/5 [00:02<00:00,  2.25it/s]


Epoch [35/36], Loss: 0.0078, Accuracy: 1.0000


100%|██████████| 5/5 [00:02<00:00,  1.88it/s]

Epoch [36/36], Loss: 0.0073, Accuracy: 1.0000


In [24]:
from sklearn.metrics import precision_score, f1_score, recall_score, accuracy_score

In [26]:
# Evaluate the model on the test dataset
model.eval()
with torch.no_grad():
    run_loss = 0.0
    run_acc = 0.0
    y_true = []
    y_pred = []
    for featur, labels in test_loader:
        featur = featur.to(device)
        labels = labels.to(device)
        out = model(featur)
        loss = crit(out, labels)
        run_loss += loss.item() * featur.size(0)
        _, preds = torch.max(out, 1)
        run_acc += torch.sum(preds == labels.data)
        y_true += labels.cpu().numpy().tolist()
        y_pred += preds.cpu().numpy().tolist()
    test_loss = run_loss / len(test_t_set)
    test_acc = run_acc.double() / len(test_t_set)

# Calculate precision, recall, and F1 score
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
accuracy = accuracy_score(y_true, y_pred)
print("<---------Model Performance on the test dataset-------->")
print("Accuracy: ", accuracy)
print("Precision Score: ", precision)
print("Recall Score: ", recall)
print("F1 Score: ", f1)

<---------Model Performance on the test dataset-------->
Accuracy:  0.7474747474747475
Precision Score:  0.7441860465116279
Recall Score:  0.6956521739130435
F1 Score:  0.7191011235955057
